# Reactor Kinetics Example 

Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a reactor kinetics experiment with the Pyomo.DOE.
    

## Step 0: Import Pyomo and Pyomo.DOE module

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
#from scipy.interpolate import interp2d
import pandas as pd 
#from itertools import permutations, product, combinations
import idaes

from fim_doe import *

## Step 1: Import Reaction Example Mathematical Model

Consider two chemical reactions that converts molecule $A$ to desired product $B$ and a less valuable side-product $C$.

$A \overset{k_1}{\rightarrow} B \overset{k_2}{\rightarrow} C$

Our ultimate goals is to design a large-scale continous reactor that maximizes the production of $B$. This general sequential reactions problem is widely applicable to CO$_2$ capture and industry more broadly (petrochemicals, pharmasuticals, etc.).

The rate laws for these two chemical reactions are:

$r_A = -k_1 C_A$

$r_B = k_1 C_A - k_2 C_B$

$r_C = k_2 C_B$

Here, $C_A$, $C_B$, and $C_C$ are the concentrations of each species. The rate constants $k_1$ and $k_2$ depend on temperature as follows:

$k_1 = A_1 \exp{\frac{-E_1}{R T}}$

$k_2 = A_2 \exp{\frac{-E_2}{R T}}$

$A_1, A_2, E_1$, and $E_2$ are fitted model parameters. $R$ is the ideal-gas constant and $T$ is absolute temperature.

Using the **CCSI$^2$ toolset**, we would like do the following perform:

Perform **uncertainty quantification** and **design of experiments** on a small-scale **batch reactor** to infer parameters $A_1$, $A_2$, $E_1$, and $E_2$.

### Batch reactor

The concenrations in a batch reactor evolve with time per the following differential equations:

$$ \frac{d C_A}{dt} = r_A = -k_1 C_A $$

$$ \frac{d C_B}{dt} = r_B = k_1 C_A - k_2 C_B $$

$$ \frac{d C_C}{dt} = r_C = k_2 C_B $$

This is a linear system of differential equations. Assuming the feed is only species $A$, i.e., 

$$C_A(t=0) = C_{A0} \quad C_B(t=0) = 0 \quad C_C(t=0) = 0$$

When the temperature is constant, it leads to the following analytic solution:

$$C_A(t) = C_{A,0} \exp(-k_1 t)$$

$$C_B(t) = \frac{k_1}{k_2 - k_1} C_{A,0} \left[\exp(-k_1 t) - \exp(-k_2 t) \right]$$

$$C_C(t) = C_{A,0} - \frac{k_2}{k_2 - k_1} C_{A,0} \exp(-k_1 t) + \frac{k_1}{k_2 - k_1} \exp(-k_2 t) C_{A,0} = C_{A,0} - C_{A}(t) - C_{B}(t)$$

In [2]:
from reactor_kinetics import *

## Step 2: Define inputs

In [3]:
# Create model function
createmod = create_model

# discretization by Pyomo.DAE
disc = disc_for_measure

# Control time set [h]
t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
    
# Measurement time points [h]
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

# design variable and its control time set
dv_pass = {'CA0': [0],'T': t_control}
    
# Create measurement object
measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
measure_class =  Measurements(measure_pass)

# Define parameter nominal value 
parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}

All measurements are flattened.
Flatten measurement name: ['C_index_CA', 'C_index_CB', 'C_index_CC']


In [4]:
def generate_exp(t_set, CA0, T):  
    '''Generate experiments. 
    t_set: time control set for T.
    CA0: CA0 value
    T: A list of T 
    '''
    assert(len(t_set)==len(T)), 'T should have the same length as t_set'
    
    T_con_initial = {}
    for t, tim in enumerate(t_set):
        T_con_initial[tim] = T[t]
        
    dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
    return dv_dict_overall

In [5]:
# empty prior
prior_all = np.zeros((4,4))


# add prior information
#prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
#prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

#prior_all = prior_5_300


prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0. 0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Method: Compute FIM 

This method computes an MBDoE optimization problem with no Degree of Freedom.

In [6]:
# choose from 'sequential_finite', 'direct_kaug'
# 'sequential_sipopt', 'sequential_kaug' is also available
sensi_opt = 'sequential_finite'
#sensi_opt = 'direct_kaug'

# model option
if sensi_opt == 'direct_kaug':
    args_ = [False]
else:
    args_ = [True]
    

# Define experiments
exp1 = generate_exp(t_control, 5, [570, 300, 300, 300, 300, 300, 300, 300, 300])

In [7]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_)


result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', 
                                store_output = 'store_output', read_output=None,
                                formula='central')


result.calculate_FIM(doe_object.design_values)


This scenario: {'A1': {0: 84.8756493935153}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.0848756493935153, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/

Output this time:  [5.0, 0.6706243700020834, 0.4195560837359151, 0.2624827180069096, 0.16421446362737377, 0.10273586874285646, 0.06427362421801934, 0.04021087104893356, 0.02515672906555432, 0.0, 1.4356608195123035, 1.5205901724705073, 1.5079974275825079, 1.4413335440984965, 1.347049388125238, 1.241037622436591, 1.1326547091791959, 1.027229845637414, 0.0, 2.8937148104856125, 3.0598537437935778, 3.229519854410583, 3.39445199227413, 3.5502147431319058, 3.69468875334539, 3.8271344197718706, 3.947613425297032]
This scenario: {'A1': {0: 84.79085853498033}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.784809060054453}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479085853498033, 'A2': 0.37171773413976417, 'E1': 0.007784809060054453, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************

Output this time:  [5.0, 0.6706243700020836, 0.4195560837359153, 0.2624827180069097, 0.16421446362737382, 0.10273586874285653, 0.06427362421801941, 0.04021087104893361, 0.02515672906555436, 0.0, 1.441416665040551, 1.5268448073993228, 1.514721262165308, 1.4484455916701346, 1.3544479171661528, 1.248617623361608, 1.1403173456620836, 1.03488709459508, 0.0, 2.887958964957366, 3.0535991088647614, 3.222796019827782, 3.3873399447024912, 3.542816214090991, 3.687108752420372, 3.819471783288983, 3.939956176339366]
This scenario: {'A1': {0: 84.70606767644534}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08470606767644534, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
T

Output this time:  [5.0, 0.6706243700020837, 0.4195560837359155, 0.26248271800690987, 0.16421446362737402, 0.10273586874285656, 0.06427362421801945, 0.04021087104893362, 0.02515672906555435, 0.0, 1.4384112269396816, 1.5233649787335601, 1.5108005401811877, 1.4441529235617967, 1.3498659292064588, 1.243830257365472, 1.135403210540418, 1.029916244982974, 0.0, 2.890964403058235, 3.0570789375305245, 3.2267167418119023, 3.3916326128108296, 3.547398202050685, 3.691896118416508, 3.8243859184106483, 3.9449270259514715]
This scenario: {'A1': {0: 84.79085853498033}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.769254995998401}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479085853498033, 'A2': 0.37171773413976417, 'E1': 0.0077692549959984016, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


*************************************************************************

Output this time:  [5.0, 0.670624370002084, 0.41955608373591563, 0.2624827180069099, 0.16421446362737396, 0.10273586874285658, 0.06427362421801945, 0.040210871048933616, 0.02515672906555434, 0.0, 1.4326573615440616, 1.5171119900116485, 1.5040785376836843, 1.4370435698520607, 1.342471604700488, 1.2362565062752762, 1.1277492573498347, 1.0222703526130221, 0.0, 2.8967182684538546, 3.0633319262524363, 3.2334387443094053, 3.398741966520566, 3.554792526556655, 3.699469869506704, 3.8320398716012316, 3.9525729183214233]
Build time with sequential_finite mode [s]: 0.0396122932434082
Solve time with sequential_finite mode [s]: 0.9610984325408936
Total wall clock time [s]: 1.5833582878112793
Existed information has been added.
FIM: [[ 2.45738410e-03  5.94863782e-04 -4.56435589e-02 -6.00876165e-02]
 [ 5.94863782e-04  2.22810780e-04 -1.03935357e-02 -2.23273539e-02]
 [-4.56435589e-02 -1.03935357e-02  8.61211473e-01  1.05437230e+00]
 [-6.00876165e-02 -2.23273539e-02  1.05437230e+00  2.27036487e+00]]
T

In [8]:
mod_eg = doe_object.models[0]

t_list = []
for t in mod_eg.t:
    t_list.append(t)
    
print('kp1:')
for i in t_list:
    print(value(mod_eg.kp1[0,i]))
    
print('kp2:')
for i in t_list:
    print(value(mod_eg.kp2[0,i]))
    
print('CA:')
for i in t_list:
    print(value(mod_eg.C[0,'CA',i]))
    
print('CB:')
for i in t_list:
    print(value(mod_eg.C[0,'CB',i]))
    
print('CC:')
for i in t_list:
    print(value(mod_eg.C[0,'CC',i]))

kp1:
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.75584

In [9]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

======Result summary======
Four design criteria log10() value:
A-optimality: 0.49613453997436285
D-optimality: -10.3118203437022
E-optimality: -5.553641798357302
Modified E-optimality: 6.006360465080967


## Method: Optimization
Gradient-based optimization with IPOPT with .optimize_doe()

This function solves twice: It solves the square version of the MBDoE problem firstly, and then unfixes the design variables as Degree of Freedoms and solves again. In this way the optimization problem can be well initialized. 

In [10]:
#exp1 = generate_exp(t_control, 5, [477.43, 300, 300, 300, 300, 300, 300, 300, 300])
exp1 = generate_exp(t_control, 5, [477.43, 300, 300, 300, 300, 300, 300, 300, 300])

In [12]:
from itertools import permutations, product
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=[True])

square_result, optimize_result= doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=None)

#square_result = doe_object.optimize_doe(exp1, if_optimize=False, if_Cholesky=True, 
#                                                         scale_nominal_param_value=True, objective_option='det', 
#                                                         L_initial=None)

Sensitivity information is scaled by its corresponding parameter nominal value.
    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2

Reallocating memory for MA57: lfact (513180)
Reallocating memory for MA57: lfact (962390)
Reallocating memory for MA57: lfact (1058577)
  11  1.3743846e+01 8.41e+05 9.90e+06  -1.0 2.76e+04  -0.0 3.96e-04 4.49e-04h  1
Reallocating memory for MA57: lfact (2148001)
Reallocating memory for MA57: lfact (2480350)
  12  1.4067657e+01 8.41e+05 1.30e+07  -1.0 6.79e+04   0.4 3.60e-04 1.05e-04h  1
  13r 1.4067657e+01 8.41e+05 9.99e+02   5.2 0.00e+00   0.8 0.00e+00 3.43e-07R  4
  14r 6.2870147e+00 8.41e+05 2.22e+04   5.2 1.61e+08    -  5.15e-04 2.26e-05f  1
  15r 1.2703380e+01 8.40e+05 6.15e+04   4.5 1.10e+08    -  2.29e-01 4.80e-04f  1
  16  1.2622044e+01 8.26e+05 9.01e+02  -1.0 6.47e+05    -  2.08e-02 1.74e-02h  1
  17  1.2621332e+01 8.26e+05 9.01e+02  -1.0 3.82e+05    -  3.90e-02 2.10e-04h  1
  18  1.3775121e+01 7.35e+05 3.15e+05  -1.0 1.23e+06    -  2.63e-03 1.11e-01h  1
  19  2.2985944e+01 7.35e+05 3.14e+05  -1.0 1.50e+04   0.4 4.77e-04 7.18e-05h  1
iter    objective    inf_pr   inf_du lg(mu)

 106 -1.3503480e+01 9.59e+06 2.22e+05  -1.0 3.92e+03  -3.9 9.61e-02 2.81e-02w  1
 107 -1.3496035e+01 9.54e+06 2.21e+05  -1.0 6.91e+03  -4.3 2.04e-02 5.67e-03w  1
 108 -1.7757875e+01 1.41e+05 2.42e+05  -1.0 2.53e+03  -3.9 1.00e+00 5.88e-05h 11
 109 -1.7767475e+01 1.41e+05 2.42e+05  -1.0 4.82e+04    -  5.91e-02 1.11e-03h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110 -1.7776588e+01 1.41e+05 2.42e+05  -1.0 4.73e+05    -  4.18e-02 1.75e-04h 12
 111 -1.7784699e+01 1.41e+05 2.42e+05  -1.0 5.08e+05    -  1.70e-01 8.53e-05h 13
 112 -1.7790213e+01 1.41e+05 2.42e+05  -1.0 5.45e+05    -  1.73e-02 4.17e-05h 14
 113 -1.7803351e+01 1.41e+05 2.42e+05  -1.0 6.06e+05    -  4.47e-01 6.42e-05h 13
 114 -1.7803682e+01 1.41e+05 2.41e+05  -1.0 6.96e+03  -4.4 5.24e-02 1.10e-04h  8
 115 -1.7804049e+01 1.41e+05 2.41e+05  -1.0 7.13e+03  -4.9 1.05e-01 5.11e-05h  9
 116 -1.7714813e+01 1.40e+05 2.40e+05  -1.0 5.45e+03  -3.5 4.99e-03 4.99e-03s 15
 117 -1.7717883e+01 1.40e+05

 202 -7.7902829e+00 9.94e+04 1.08e+05  -1.0 7.51e+04  -5.9 7.08e-02 3.42e-04h 11
 203 -7.8004179e+00 9.94e+04 1.08e+05  -1.0 1.88e+05  -6.4 4.54e-02 1.35e-04h 11
 204 -7.8097401e+00 9.94e+04 1.08e+05  -1.0 8.13e+04  -6.0 3.09e-01 3.53e-04h 11
 205 -7.9787159e+00 9.93e+04 1.08e+05  -1.0 7.04e+05  -6.4 7.42e-03 6.82e-04h  7
 206 -7.9823683e+00 9.93e+04 1.08e+05  -1.0 6.76e+04  -6.0 1.28e-01 2.25e-04h 11
 207 -7.9839690e+00 9.93e+04 1.08e+05  -1.0 2.72e+04  -5.6 9.22e-02 2.20e-04h 11
 208 -7.9858182e+00 9.93e+04 1.08e+05  -1.0 9.83e+04  -6.1 8.99e-02 7.40e-05h 12
 209 -7.9854514e+00 9.93e+04 1.08e+05  -1.0 3.74e+04  -5.6 6.75e-01 2.03e-04h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 210 -7.9801636e+00 9.92e+04 1.08e+05  -1.0 1.78e+05  -6.1 1.30e-01 2.43e-04h  8
 211r-7.9801636e+00 9.92e+04 1.00e+03   4.6 0.00e+00  -5.7 0.00e+00 3.00e-07R 20
 212r-1.0573609e+01 9.92e+04 9.98e+02   4.6 3.74e+07    -  1.08e-03 9.98e-04f  1
 213 -1.7545816e+00 9.18e+04

 296 -1.8715497e+01 4.86e+04 1.85e+06  -1.0 1.74e+03   0.0 8.21e-02 2.08e-03h  5
 297 -1.8777885e+01 4.86e+04 1.85e+06  -1.0 9.40e+03  -0.4 1.55e-03 8.93e-05h  6
 298 -1.8903843e+01 4.85e+04 1.85e+06  -1.0 2.07e+03  -0.0 1.45e-01 8.55e-04h  6
 299 -1.9084070e+01 4.85e+04 1.85e+06  -1.0 7.31e+03  -0.5 2.14e-03 2.10e-04h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 300 -1.9277120e+01 4.85e+04 1.86e+06  -1.0 2.27e+03  -0.1 8.29e-02 9.80e-04h  7
 301 -1.7295321e+01 8.59e+04 9.75e+06  -1.0 1.23e+04  -0.5 1.57e-03 1.67e-02w  1
 302 -1.8024500e+01 6.25e+04 7.33e+07  -1.0 7.04e+02  -0.1 4.58e-01 2.83e-01w  1
 303 -1.8062598e+01 6.22e+04 7.29e+07  -1.0 9.51e+03  -0.6 1.31e-02 4.75e-03w  1
 304 -1.9605272e+01 4.85e+04 1.86e+06  -1.0 2.19e+05    -  1.57e-03 2.61e-04h  6
 305 -1.9749943e+01 4.84e+04 1.86e+06  -1.0 2.95e+03  -0.2 3.68e-01 6.14e-04h  8
 306 -1.9836233e+01 4.84e+04 1.88e+06  -1.0 1.70e+03   0.3 2.71e-02 8.45e-04h  9
 307 -2.0050790e+01 4.84e+04

...

1629 -2.0659329e+01 4.04e+06 5.43e+11  -1.0 2.08e+05   5.7 5.46e-04 3.00e-03w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1630 -3.7793928e+01 6.30e+05 3.01e+01  -1.0 2.12e+05   6.1 1.00e+00 1.95e-03h  9
1631 -3.7794632e+01 6.29e+05 3.02e+01  -1.0 4.61e+06    -  6.75e-01 1.95e-03h 10
1632 -3.7795353e+01 6.28e+05 3.02e+01  -1.0 4.59e+06    -  1.00e+00 1.95e-03h 10
1633 -3.7796082e+01 6.26e+05 3.03e+01  -1.0 4.57e+06    -  6.76e-01 1.95e-03h 10
1634 -3.7796826e+01 6.25e+05 3.04e+01  -1.0 4.56e+06    -  1.00e+00 1.95e-03h 10
1635 -3.7797578e+01 6.24e+05 3.04e+01  -1.0 4.54e+06    -  6.76e-01 1.95e-03h 10
1636 -3.7798346e+01 6.23e+05 3.05e+01  -1.0 4.52e+06    -  1.00e+00 1.95e-03h 10
1637 -3.7799122e+01 6.21e+05 3.05e+01  -1.0 4.50e+06    -  6.76e-01 1.95e-03h 10
1638 -3.7799912e+01 6.20e+05 3.06e+01  -1.0 4.49e+06    -  1.00e+00 1.95e-03h 10
1639 -3.7800710e+01 6.19e+05 3.06e+01  -1.0 4.47e+06    -  6.76e-01 1.95e-03h 10
iter    objective    inf_pr 

1725 -3.7437894e+01 5.19e+05 7.08e+03  -1.0 2.24e+04  -0.6 1.13e-01 6.70e-04h  1
1726 -3.7724082e+01 5.18e+05 4.63e+04  -1.0 2.25e+04  -1.1 9.72e-03 1.49e-03h  1
1727 -4.2023598e+01 5.61e+05 1.55e+06  -1.0 2.25e+04  -1.6 7.95e-03 5.12e-02h  1
1728 -3.8385704e+01 6.45e+05 2.74e+06  -1.0 2.20e+04  -2.0 1.13e-02 8.32e-02h  1
1729 -3.7949208e+01 6.45e+05 5.24e+06  -1.0 2.34e+04  -0.7 2.13e-02 4.91e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1730 -3.7845268e+01 6.45e+05 3.98e+07  -1.0 2.49e+04  -0.3 4.90e-02 8.87e-05h  1
1731 -3.7698557e+01 6.45e+05 8.57e+07  -1.0 2.53e+04   0.1 2.07e-02 6.25e-05h  1
1732 -3.6329758e+01 6.44e+05 1.59e+08  -1.0 2.60e+04  -0.3 1.03e-01 2.90e-04h  2
1733 -3.6189956e+01 6.33e+05 1.57e+08  -1.0 2.37e+04  -0.8 2.28e-02 1.84e-02h  1
1734 -3.6190337e+01 6.33e+05 1.74e+08  -1.0 1.52e+05  -1.3 4.98e-02 1.28e-05h  1
1735 -2.6997415e+01 6.31e+05 1.88e+08  -1.0 6.78e+04  -1.8 8.75e-02 2.80e-03h  1
1736 -2.7025374e+01 6.33e+05

In iteration 1819, 1 Slack too small, adjusting variable bound
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1820 -3.7847247e+01 6.58e-05 8.16e+05  -8.6 1.86e-06  11.5 1.00e+00 8.18e-01h  1
In iteration 1820, 1 Slack too small, adjusting variable bound
1821 -3.7847247e+01 6.57e-05 8.15e+05  -8.6 1.99e-04  11.0 4.66e-01 8.85e-04h  1
In iteration 1821, 1 Slack too small, adjusting variable bound
1822 -3.7847247e+01 6.57e-05 1.94e+06  -8.6 2.10e-04  10.6 1.00e+00 8.47e-04h  1
In iteration 1822, 1 Slack too small, adjusting variable bound
1823 -3.7847247e+01 6.53e-05 1.93e+06  -8.6 3.39e+01    -  4.61e-03 6.12e-03h  1
1824 -3.7847247e+01 8.99e-06 5.00e+04  -8.6 4.16e-06  10.1 9.77e-01 1.00e+00h  1
In iteration 1824, 1 Slack too small, adjusting variable bound
1825 -3.7847247e+01 8.55e-06 7.49e+04  -8.6 3.25e-04   9.6 1.61e-01 4.96e-02h  1
1826 -3.7847247e+01 8.50e-06 7.68e+04  -8.6 3.09e-04   9.1 4.96e-02 5.42e-03h  2
1827 -3.7847247e+01 8.50e-06 7.68e+04

1915r-3.7847076e+01 1.16e-10 9.18e+02  -8.6 0.00e+00    -  0.00e+00 4.06e-07R 16
1916r-3.7847076e+01 1.16e-10 9.94e+00  -8.6 1.00e-01    -  9.89e-01 8.89e-02f  1
1917r-3.7847048e+01 1.10e-05 7.23e-01  -8.6 9.72e+00    -  9.27e-01 1.00e+00h  1
1918r-3.7846772e+01 8.70e-04 3.42e-01  -8.6 4.20e+01    -  5.26e-01 1.00e+00h  1
1919r-3.7846575e+01 9.90e-04 8.12e-07  -8.6 2.60e+01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1920r-3.7846485e+01 1.09e-04 3.79e-08  -8.6 1.94e+00    -  1.00e+00 1.00e+00h  1
1921r-3.7846492e+01 8.22e-07 3.51e-10  -8.6 2.58e-01    -  1.00e+00 1.00e+00h  1
1922r-3.7846492e+01 1.16e-10 1.14e-13  -8.6 2.88e-03    -  1.00e+00 1.00e+00h  1
1923r-3.7846818e+01 1.52e-03 7.03e-07 -11.0 1.32e+01    -  1.00e+00 1.00e+00f  1
1924r-3.7846836e+01 4.41e-06 2.82e-08 -11.0 6.70e-01    -  1.00e+00 1.00e+00h  1
1925r-3.7846945e+01 1.86e-08 4.84e-09 -11.0 4.50e+00    -  1.00e+00 1.00e+00H  1
1926r-3.7846974e+01 1.86e-09

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
from pyomo.core.kernel.component_set import ComponentSet

def large_residuals_set(block, tol=1e-5, LOUD=True):
    """
    Method to return a ComponentSet of all Constraint components with a
    residual greater than a given threshold which appear in a model.
    Args:
        block : model to be studied
        tol : residual threshold for inclusion in ComponentSet
    Returns:
        A ComponentSet including all Constraint components with a residual
        greater than tol which appear in block
    """
    large_residuals_set = ComponentSet()
    for c in block.component_data_objects(
            ctype=Constraint, active=True, descend_into=True):
        if c.active and value(c.lower - c.body()) > tol:
            large_residuals_set.add(c)
        elif c.active and value(c.body() - c.upper) > tol:
            large_residuals_set.add(c)
    
    if LOUD:
        print("All equality constraints with residuals larger than",tol)
        for c in large_residuals_set:
            print(c)
            
    return large_residuals_set

large_residuals_set(mod_eg, tol=1.0E-5)

In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('The result FIM is:', optimize_result.FIM)
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(optimize_result.trace))
print('D-optimality:', np.log10(optimize_result.det))
print('E-optimality:', np.log10(optimize_result.min_eig))
print('Modified E-optimality:', np.log10(optimize_result.cond))

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

print(value(optimize_result.model.T[0]))
print(value(optimize_result.model.T[1]))
print()
    
si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Method: Exploratory analysis (Enumeration)

This method conducts exploratory analysis by enumeration. 
It allows to define any number (dimensions) of design variables.
Heatmaps can be drawn by two design variables, fixing other design variables; 
1D curve can be drawn by one design variable, fixing other design variables.

### Specify user inputs

In [ ]:
# Design variable ranges as lists 
design_ranges = [list(np.linspace(1,5,5)), list(np.linspace(300,700,5))]

# Design variable names 
dv_apply_name = ['CA0','T']

# Design variable should be fixed at these time points
dv_apply_time = [[0],t_control]

# Define experiments. This is a starting point of which the value does not matter
exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])
    
## choose from 'sequential_finite', 'direct_kaug'
#sensi_opt = 'sequential_finite'
sensi_opt = 'direct_kaug'

# model option
if sensi_opt == 'direct_kaug':
    args_ = [False]
else:
    args_ = [True]

In [ ]:
# add prior information
prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
#prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

prior_all = prior_5_300

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges, dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt)

### 1D sensitivity curve

In [ ]:
test = all_fim.extract_criteria()

## draw 1D sensitivity curve 

fixed = {"'CA0'": 5.0}

all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )



## Heatmap

In [ ]:
fixed = {}
all_fim.figure_drawing(fixed, ['CA0','T'], 'Reactor case','$C_{A0}$ [M]', 'T [K]' )

## Grid search for 3 design variables

In [ ]:

# Define design ranges
design_ranges = [list(np.linspace(1,5,2)),  list(np.linspace(300,700,2)), [300,500]]

# Define design variable 
# Here the two T are for different controlling time subsets
dv_apply_name = ['CA0', 'T', 'T']
dv_apply_time = [[0], [0], [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875,1]]

# Define experiments
exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])

## choose from 'sequential_finite', 'direct_kaug'
#sensi_opt = 'sequential_finite'
sensi_opt = 'direct_kaug'

# model option
if sensi_opt == 'direct_kaug':
    args_ = [False]
else:
    args_ = [True]

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges, dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt)

### Draw 1D sensitivity curve

In [ ]:
test = all_fim.extract_criteria()

In [ ]:
## draw 1D sensitivity curve 

fixed = {"'CA0'": 1.0, "'T2'": 300}

all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )

### Draw 2D sensitivity curve

In [ ]:
fixed = {"'T2'": 300}

all_fim.figure_drawing(fixed, ['CA0','T'], 'Reactor case','$C_{A0}$ [M]', 'T [K]' )